In [1]:
import numpy as np, pandas as pd, time, os, subprocess, scipy as sp, re, seaborn as sns
import app_config, importlib, app_lib
import diffmap as dm
from sklearn.metrics.pairwise import euclidean_distances

# plot_data_df = all_data.obs
# plot_data_df[app_config.params['display_coordinates']['x']] = all_data.obsm['X_umap'][:,0]
# plot_data_df[app_config.params['display_coordinates']['y']] = all_data.obsm['X_umap'][:,1]

In [2]:
plot_data_df = pd.read_csv(app_config.params['plot_data_df_path'][0], sep="\t", index_col=False)

In [22]:
gene_names_df2 = pd.read_csv(app_config.params['gene_fullnames_path'], sep="\t", index_col=False)
a = dict(zip(gene_names_df2['symbol'], gene_names_df2['name']))
b = dict(zip(gene_names_df2['symbol'], gene_names_df2['ensembl_gene_id']))
full_point_names = np.copy(point_names)
ens_IDs = np.copy(point_names)

for i in range(len(point_names)):
    if point_names[i] in a:
        full_point_names[i] = a[point_names[i]]
        ens_IDs[i] = b[point_names[i]]

In [40]:
point_names[14]

'AAED1'

In [32]:
pd.DataFrame(full_point_names).to_csv(app_config.params['genenames_path'], header=False, index=False)
pd.DataFrame(ens_IDs).to_csv(app_config.params['gene_ensID_path'], header=False, index=False)

In [3]:
# Load gene embedded coordinates.
plot_data_df = pd.read_csv(app_config.params['plot_data_df_path'][0], sep="\t", index_col=False)
# graph_adj = sp.sparse.load_npz(app_config.params['adj_mat_path'])
data_ess = pd.read_csv(app_config.params['raw_ess_data_path'], index_col=0, header=0, sep='\t')
data_ess = data_ess[data_ess.columns[:-4]]   # Only first 481 cols put through GLS, so isolate these

point_names = np.array(plot_data_df['gene_names'])
feat_names = data_ess.columns
cancer_types = data_ess.columns.str.split('_').str[1:].str.join(' ').str.capitalize().str.replace('Haematopoietic and lymphoid tissue', 'Hematopoietic/lymphoid')
additional_colorvars = []#app_config.params['additional_colorvars']

raw_data = data_ess.values

In [5]:
itime = time.time()
print(time.time() - itime)
mutation_df = pd.read_csv(app_config.params['mutation_data_path'], sep="\t", index_col=False)
print(time.time() - itime)
shRNA_df = pd.read_csv(app_config.params['shRNA_data_path'], sep=",", index_col=0)
print(time.time() - itime)
expr_df = pd.read_csv(app_config.params['expression_data_path'], sep="\t", skiprows=2, index_col=False)
print(time.time() - itime)

3.719329833984375e-05


/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning:

Columns (3,25,26,27,30,31) have mixed types. Specify dtype option on import or set low_memory=False.



6.320101261138916
8.934747219085693
24.0143461227417


In [104]:
geneview_data = mutation_df
mutdata = geneview_data[geneview_data[:, 0] == 'TP53', :]

TypeError: unhashable type: 'slice'

In [105]:
mutation_df.columns   #[['dbSNP_Val_Status', 'Genome_Change', 'Annotation_Transcript', 'Tumor_Sample_Barcode', 'cDNA_Change', 'Codon_Change', 'Protein_Change', 'isDeleterious', 'isTCGAhotspot', 'TCGAhsCnt', 'isCOSMIChotspot', 'COSMIChsCnt', 'ExAC_AF', 'WES_AC', 'SangerWES_AC']]

Index(['Hugo_Symbol', 'Entrez_Gene_Id', 'NCBI_Build', 'Chromosome',
       'Start_position', 'End_position', 'Strand', 'Variant_Classification',
       'Variant_Type', 'Reference_Allele', 'Tumor_Seq_Allele1', 'dbSNP_RS',
       'dbSNP_Val_Status', 'Genome_Change', 'Annotation_Transcript',
       'Tumor_Sample_Barcode', 'cDNA_Change', 'Codon_Change', 'Protein_Change',
       'isDeleterious', 'isTCGAhotspot', 'TCGAhsCnt', 'isCOSMIChotspot',
       'COSMIChsCnt', 'ExAC_AF', 'WES_AC', 'SangerWES_AC',
       'SangerRecalibWES_AC', 'RNAseq_AC', 'HC_AC', 'RD_AC', 'WGS_AC',
       'Broad_ID'],
      dtype='object')

In [71]:
importlib.reload(app_config)
### noexpr_genes = point_names[np.where(~np.isin(point_names, expr_df['Description']))[0]]
# expr_arr = np.array(expr_df.iloc[:, 1:])
# arr_ndces = np.isin(expr_arr[:,0], point_names)
# np.save(app_config.params['expression_arr_path'], expr_arr[arr_ndces,:])
# np.save(app_config.params['expression_cell_lines_path'], [c.split(' (ACH')[0] for c in expr_df.columns[2:]])
# expr_arr = np.load(app_config.params['expression_arr_path'])
# expr_cell_lines = np.load(app_config.params['expression_cell_lines_path'])
# np.save(app_config.params['expression_arr_path'], np.vstack((np.array(['Gene'] + list(expr_cell_lines)), expr_arr)))

In [85]:
#expr_arr = np.load(app_config.params['expression_arr_path'])
exprdata_celllines = expr_arr[0, :]#[expr_arr[:, 0] == 'TP53', :]

In [87]:
np.array(expr_arr[:, exprdata_celllines == 'T24_URINARY_TRACT']).flatten()

array(['T24_URINARY_TRACT', 0.0, 0.0, ..., 6.9847, 0.0, 3.97005],
      dtype=object)

In [131]:
row_ndces = []
for g in point_names:
    if g in expr_df['Description']:
        row_ndces.append(np.where(expr_df['Description'] == g)[0][0])
col_ndces = []
for g in feat_names:
    if g in expr_df['Description']:
        col_ndces.append(np.where(expr_df['Description'] == g)[0][0])

(56318,)

In [97]:
nornai_feats = feat_names[np.where(~np.isin(feat_names, shRNA_df.columns))[0]]
nornai_genes = point_names[np.where(~np.isin(point_names, [c.split(' (')[0] for c in shRNA_df.index]))[0]]

In [106]:
# Save an abbreviated version of the mutation data, for speed reasons.
mutation_arr = np.array(mutation_df[['Hugo_Symbol', 'Chromosome', 'Start_position', 'End_position', 'Variant_Classification', 'Variant_Type', 
                                     'Tumor_Sample_Barcode', 'isDeleterious', 'isTCGAhotspot', 'isCOSMIChotspot', 'Codon_Change', 'Protein_Change']])
np.save(app_config.params['mutation_arr_path'], mutation_arr)

In [114]:
mutdata = mutation_arr[mutation_arr[:, 0] == 'TP53', :]
mutdata[:, [1,2,4,10,11]]
#where_mutations = np.isin(feat_names, xd)

array([['17', 7576854, 'Splice_Site', 'c.(991-993)cAg>cGg', 'p.Q331R'],
       ['17', 7577120, 'Missense_Mutation', 'c.(817-819)cGt>cAt',
        'p.R273H'],
       ['17', 7577536, 'Missense_Mutation', 'c.(745-747)Agg>Ggg',
        'p.R249G'],
       ...,
       ['17', 7577150, 'Splice_Site', 'c.(781-789)aggtagtgg>agg',
        'p.*W262fs'],
       ['17', 7578479, 'Frame_Shift_Del', 'c.(448-453)acacccfs',
        'p.P153fs'],
       ['17', 7578483, 'Frame_Shift_Del', 'c.(445-447)tccfs', 'p.S149fs']],
      dtype=object)

In [83]:
query_results = mutdata[(mutdata[:, 6] == feat_names[143]), :]
qwe = ["chr{}:{}    {}".format(
    query_results[i, 1], query_results[i, 2], query_results[i, 4]
    #, query_results[i, 7], query_results[i, 8], query_results[i, 9]
) for i in range(query_results.shape[0])]

In [23]:
gvdata = np.zeros_like(feat_names)
for i in range(len(feat_names)):
    if where_mutations[i]:
        gvdata[i] = 
# gvdata[np.isin(feat_names, xd)]

(331,)

# GO enrichment

In [46]:
from goatools.associations import read_ncbi_gene2go
from goatools.base import download_ncbi_associations
# Get ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz
gene2go = download_ncbi_associations()
go2geneids_human = read_ncbi_gene2go("gene2go", taxids=[9606], go2geneids=True)

from goatools.go_search import GoSearch
go2geneids_human = read_ncbi_gene2go("/Users/akshay/github/coessentiality-browser/gene2go", taxids=[9606], go2geneids=True)
srchhelp = GoSearch("data/go-basic.obo", go2items=go2geneids_human)

  wget.download(ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz out=gene2go.gz)


  gunzip gene2go.gz
  18,069 items READ: gene2go


In [102]:
gos = srchhelp.get_matching_gos(re.compile('0000003'))
mn = srchhelp.get_items(gos)

from goatools.test_data.genes_NCBI_9606_ProteinCoding import GENEID2NT


PATTERN SEARCH: "0000003"
MATCH GO:0030509(BMP signaling pathway) defn: "A series of molecular signals initiated by the binding of a member of the BMP (bone morphogenetic protein) family to a receptor on the surface of a target cell, and ending with regulation of a downstream cellular process, e.g. transcription." [GOC:signaling, ISBN:0878932437, PR:000000034]
MATCH GO:0036122(BMP binding) defn: "Interacting selectively and non-covalently with a member of the bone morphogenetic protein (BMP) family." [GOC:BHF, PMID:9660951, PR:000000034]
Found 2 GO(s) out of 18069 matching pattern("0000003")


In [121]:
from goatools import obo_parser
go_obo = 'data/go-basic.obo'
go = obo_parser.GODag(go_obo)
gotermIDs = []
gotermnames = []
for go_term in go.values():
    gotermIDs.append(str(go_term.id))
    gotermnames.append(str(go_term.name))

data/go-basic.obo: fmt(1.2) rel(2018-09-19) 47,343 GO Terms


In [122]:
np.save('gotermIDs.npy', np.array(gotermIDs))    
np.save('gotermnames.npy', np.array(gotermnames))


In [67]:
import goterm_caller
importlib.reload(gprofiler_caller)
gprofiler_caller.gprofiler(['GO:0000003'], mode='gconvert')
g = goterm_caller.gprofiler(['Klf4', 'Pax5', 'Sox2', 'Nanog', 'Jund', 'Max'])
g

AttributeError: 'NoneType' object has no attribute 'name'

In [57]:
ctypes = [x for x in np.unique(cancer_types)]
colorlist = ["#f7ff00","#fabebe","#ff8300","#f000ff","#74ee15","#4363d8","#001eff","#ff3300","#cab2d6","#008080","#808000","#bcf60c","#bec1d4","#fb9a99","#ffd8b1","#3cb44b","#bc5300","#ffe119","#33ccff","#911eb4","#46f0f0","#d220c8","#e6beff","#e6194b","#aaffc3","#000075"]
dict([x for x in zip(ctypes, colorlist[0:len(ctypes)])])

{'Autonomic ganglia': '#f7ff00',
 'Biliary tract': '#fabebe',
 'Bone': '#ff8300',
 'Breast': '#f000ff',
 'Central nervous system': '#74ee15',
 'Cervix': '#4363d8',
 'Endometrium': '#001eff',
 'Hematopoietic/lymphoid': '#ff3300',
 'Kidney': '#cab2d6',
 'Large intestine': '#008080',
 'Liver': '#808000',
 'Lung': '#bcf60c',
 'Oesophagus': '#bec1d4',
 'Ovary': '#fb9a99',
 'Pancreas': '#ffd8b1',
 'Pleura': '#3cb44b',
 'Skin': '#bc5300',
 'Soft tissue': '#ffe119',
 'Stomach': '#33ccff',
 'Thyroid': '#911eb4',
 'Upper aerodigestive tract': '#46f0f0',
 'Urinary tract': '#d220c8'}